In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
def collect_and_clean_files(directory):
    all_files = []
    deleted_files = []

    # Walk through all directories and files
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            
            # Check if file name contains the pattern ".txt_"
            if '.txt_' in file:
                try:
                    os.remove(file_path)
                    deleted_files.append(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Could not delete {file_path}: {e}")
            else:
                all_files.append(file_path)

    print(f"\nTotal files deleted: {len(deleted_files)}")
    return all_files

In [3]:
# Replace values according to specified rules
def replace_values(value):
    if value == '0':
        return '48'
    elif value == '-':
        return '45'
    elif value == '+':
        return '43'
    elif value == '.':
        return '45'
    else:
        try:
            num = float(value)
            if num < 0:
                return '45' + str(int(abs(num))).replace('.', '45')  # For negative values
            else:
                return '43' + str(int(num)).replace('.', '45')  # For positive values
        except ValueError:
            return value  # Return the original value if not convertible

In [4]:
# Calculate normalized Shannon entropy
def calculate_entropy(values):
    n = len(values)
    if n == 0:
        return 0
    count = Counter(values)
    probabilities = [freq / n for freq in count.values()]
    
    entropy = -sum(p * np.log2(p) for p in probabilities if p > 0)
    max_entropy = np.log2(len(count)) if len(count) > 1 else 1
    return entropy / max_entropy

In [5]:
# Convert ASCII values to binary
def convert_to_binary(ascii_values):
    return [bin(int(value))[2:] for value in ascii_values]  # Binary without '0b' prefix

In [6]:
# Write values to a file
def write_to_file(filename, values):
    with open(filename, 'w') as file:
        file.write(' '.join(values) + ' ')

In [7]:
# Process a single file: convert to ASCII, then to binary, calculate entropy
def process_file(file_path):
    ascii_values = []
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        values = line.strip().split()
        ascii_values.extend([replace_values(value) for value in values])

    # Write ASCII values to intermediate file
    ascii_filename = f'{file_path}_ascii.txt'
    write_to_file(ascii_filename, ascii_values)
    
    # Convert ASCII values to binary and write to another intermediate file
    binary_values = convert_to_binary(ascii_values)
    binary_filename = f'{file_path}_binary.txt'
    write_to_file(binary_filename, binary_values)
    
    # Calculate entropy of binary values
    entropy = calculate_entropy(binary_values)
    return round(entropy, 2)

In [8]:
# Gather files from the specified directory
def get_files_in_folder(folder_path):
    files = []
    for root, _, files_list in os.walk(folder_path):
        for file in files_list:
            if not file.endswith('.dat'):  # Ignore .dat files
                files.append(os.path.join(root, file))
    return files

In [9]:
# Gather folders within the specified main folder
def get_folders_in_folder(folder_path):
    return [os.path.join(folder_path, item) for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item))]

In [10]:
# Process each individual and period folder, and store entropies in a DataFrame
def process_individual_folders(main_folder, period_folders):
    individuals = get_folders_in_folder(main_folder)
    
    for individual in individuals:
        individual_name = os.path.basename(individual)
        print("Processing individual:", individual_name)

        # Prepare a DataFrame to store entropy results for this individual
        entropy_data = {period: {channel: None for channel in CHANNELS_MAPPING} for period in period_folders}
        
        for period_folder in period_folders:
            files = get_files_in_folder(os.path.join(individual, period_folder))
            print("Processing folder:", period_folder)
            for i, file_path in enumerate(files):
                entropy = process_file(file_path)
                channel = CHANNELS_MAPPING[i]
                entropy_data[period_folder][channel] = entropy

        # Convert to a DataFrame
        entropy_df = pd.DataFrame(entropy_data)

        # Save the DataFrame as a CSV file in the individual's folder
        save_path = os.path.join(individual, f'{individual_name}_entropy_results.csv')
        entropy_df.to_csv(save_path, index_label="Channel")
        print(f"Saved entropy results for {individual_name} to {save_path}\n")

In [11]:
collect_and_clean_files('.')

Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C1.txt_ascii.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C1.txt_binary.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C15.txt_ascii.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C15.txt_binary.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C17.txt_ascii.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C17.txt_binary.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C21.txt_ascii.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C21.txt_binary.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C5.txt_ascii.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\GENERAL\C5.txt_binary.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\ICTAL\C1.txt_ascii.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\ICTAL\C1.txt_binary.txt
Deleted: .\Patients 00 - 17\INDIVIDUO_EDF_PN00-1\ICTAL\C15.txt_ascii.txt
Deleted: .\Patients 00 - 17\IND

['.\\.gitignore',
 '.\\General_pre_post_ictal.xlsx',
 '.\\LICENSE',
 '.\\Patients 00 - 17.pdf',
 '.\\README.md',
 '.\\shannon_entropy.ipynb',
 '.\\.git\\COMMIT_EDITMSG',
 '.\\.git\\config',
 '.\\.git\\description',
 '.\\.git\\HEAD',
 '.\\.git\\index',
 '.\\.git\\packed-refs',
 '.\\.git\\hooks\\applypatch-msg.sample',
 '.\\.git\\hooks\\commit-msg.sample',
 '.\\.git\\hooks\\fsmonitor-watchman.sample',
 '.\\.git\\hooks\\post-update.sample',
 '.\\.git\\hooks\\pre-applypatch.sample',
 '.\\.git\\hooks\\pre-commit.sample',
 '.\\.git\\hooks\\pre-merge-commit.sample',
 '.\\.git\\hooks\\pre-push.sample',
 '.\\.git\\hooks\\pre-rebase.sample',
 '.\\.git\\hooks\\pre-receive.sample',
 '.\\.git\\hooks\\prepare-commit-msg.sample',
 '.\\.git\\hooks\\push-to-checkout.sample',
 '.\\.git\\hooks\\sendemail-validate.sample',
 '.\\.git\\hooks\\update.sample',
 '.\\.git\\info\\exclude',
 '.\\.git\\logs\\HEAD',
 '.\\.git\\logs\\refs\\heads\\main',
 '.\\.git\\logs\\refs\\remotes\\origin\\HEAD',
 '.\\.git\\logs\

In [12]:
# Mapping for channel identifiers
CHANNELS_MAPPING = [
    'Fp1',   # Channel 1 (Fp1)
    'O1',    # Channel 5 (O1)
    'Cz',    # Channel 15 (Cz)
    'Fp2',   # Channel 17 (Fp2)
    'O2'     # Channel 21 (O2)
]

# Main function to run the entire processing pipeline
def main():
    main_folder = 'Patients 00 - 17'
    period_folders = ['GENERAL', 'ICTAL', 'POST_ICTAL', 'PRE_ICTAL']
    
    process_individual_folders(main_folder, period_folders)

# Run the main function
if __name__ == '__main__':
    main()

Processing individual: INDIVIDUO_EDF_PN00-1
Processing folder: GENERAL
Processing folder: ICTAL
Processing folder: POST_ICTAL
Processing folder: PRE_ICTAL
Saved entropy results for INDIVIDUO_EDF_PN00-1 to Patients 00 - 17\INDIVIDUO_EDF_PN00-1\INDIVIDUO_EDF_PN00-1_entropy_results.csv

Processing individual: INDIVIDUO_EDF_PN01
Processing folder: GENERAL
Processing folder: ICTAL
Processing folder: POST_ICTAL
Processing folder: PRE_ICTAL
Saved entropy results for INDIVIDUO_EDF_PN01 to Patients 00 - 17\INDIVIDUO_EDF_PN01\INDIVIDUO_EDF_PN01_entropy_results.csv

Processing individual: INDIVIDUO_EDF_PN03
Processing folder: GENERAL
Processing folder: ICTAL
Processing folder: POST_ICTAL
Processing folder: PRE_ICTAL
Saved entropy results for INDIVIDUO_EDF_PN03 to Patients 00 - 17\INDIVIDUO_EDF_PN03\INDIVIDUO_EDF_PN03_entropy_results.csv

Processing individual: INDIVIDUO_EDF_PN05
Processing folder: GENERAL
Processing folder: ICTAL
Processing folder: POST_ICTAL
Processing folder: PRE_ICTAL
Saved en